In [18]:
# ============================================================
# 完全修正版：正しいログインフローで実装
# ============================================================

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

def setup_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless=new')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--remote-debugging-port=9222')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(30)
    driver.implicitly_wait(10)

    print("✓ Chromeドライバー初期化成功")
    return driver

def login_to_terakoya(driver, email, password):
    """
    正しいフローでログイン
    """
    print("\n" + "=" * 60)
    print("ログイン処理を開始...")
    print("=" * 60)

    try:
        # Step 1: トップページにアクセス
        top_url = "https://terakoya.sejuku.net/"
        print(f"✓ トップページにアクセス: {top_url}")
        driver.get(top_url)
        time.sleep(3)

        driver.save_screenshot("step1_top_page.png")
        print(f"  現在のURL: {driver.current_url}")

        # Step 2: ヘッダーの「ログイン」ボタンをクリック
        print("\n✓ ヘッダーの「ログイン」ボタンを検索中...")
        login_header_button = None

        # 方法1: 提供されたクラス名で探す
        try:
            login_header_button = driver.find_element(By.CSS_SELECTOR, 'div.sc-jytpVa.sc-pYNGo.ljsdtk.hsShUO')
            print("  → divクラス名で発見")
        except:
            pass

        # 方法2: テキストで探す（div要素）
        if not login_header_button:
            try:
                login_header_button = driver.find_element(By.XPATH, "//div[contains(text(), 'ログイン')]")
                print("  → divテキストで発見")
            except:
                pass

        # 方法3: すべての要素から探す
        if not login_header_button:
            try:
                all_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'ログイン')]")
                for elem in all_elements:
                    if elem.text.strip() == 'ログイン' and elem.is_displayed():
                        login_header_button = elem
                        print(f"  → {elem.tag_name}要素で発見")
                        break
            except:
                pass

        if not login_header_button:
            print("❌ ヘッダーのログインボタンが見つかりません")
            return False

        # クリック
        driver.execute_script("arguments[0].scrollIntoView(true);", login_header_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", login_header_button)
        print("✓ ヘッダーの「ログイン」をクリック")

        # ログインページへの遷移を待機
        time.sleep(3)
        driver.save_screenshot("step2_login_page.png")
        print(f"  現在のURL: {driver.current_url}")

        # Step 3: メールアドレス入力
        print("\n✓ メールアドレス入力欄を検索中...")
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="email"]'))
        )
        email_input.clear()
        time.sleep(0.5)
        email_input.send_keys(email)
        print(f"✓ メールアドレス入力完了: {email}")

        # Step 4: パスワード入力
        print("✓ パスワード入力欄を検索中...")
        password_input = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
        password_input.clear()
        time.sleep(0.5)
        password_input.send_keys(password)
        print("✓ パスワード入力完了: ********")

        time.sleep(2)
        driver.save_screenshot("step3_filled_form.png")

        # Step 5: ログイン送信ボタンをクリック
        print("\n✓ ログイン送信ボタンを検索中...")
        login_submit_button = None

        # 方法1: buttonタグで「ログイン」
        try:
            login_submit_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'ログイン')]"))
            )
            print("  → buttonタグで発見")
        except:
            pass

        # 方法2: divタグで「ログイン」（フォーム内）
        if not login_submit_button:
            try:
                divs = driver.find_elements(By.XPATH, "//div[contains(text(), 'ログイン')]")
                for div in divs:
                    # フォーム内のdivを探す
                    if div.is_displayed() and div.text.strip() == 'ログイン':
                        login_submit_button = div
                        print("  → divタグで発見")
                        break
            except:
                pass

        # 方法3: すべてのbuttonとdivから探す
        if not login_submit_button:
            buttons = driver.find_elements(By.TAG_NAME, "button")
            for btn in buttons:
                if "ログイン" in btn.text and btn.is_displayed():
                    login_submit_button = btn
                    print("  → button検索で発見")
                    break

        if not login_submit_button:
            print("❌ ログイン送信ボタンが見つかりません")
            driver.save_screenshot("error_no_submit_button.png")
            return False

        # クリック
        driver.execute_script("arguments[0].scrollIntoView(true);", login_submit_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", login_submit_button)
        print("✓ ログイン送信ボタンをクリック")

        # ログイン完了待機
        print("\n⏳ ログイン処理待機中...")
        time.sleep(6)

        driver.save_screenshot("step4_after_login.png")
        print(f"✓ ログイン後のURL: {driver.current_url}")
        print("✓ ログイン成功！")

        return True

    except Exception as e:
        print(f"\n❌ ログインエラー: {str(e)}")
        try:
            driver.save_screenshot("error_login.png")
        except:
            pass
        import traceback
        traceback.print_exc()
        return False

def navigate_to_profile(driver):
    """
    プロフィールページに移動
    """
    print("\n" + "=" * 60)
    print("プロフィールページに移動...")
    print("=" * 60)

    # ログイン後なら /account/profile にアクセス可能なはず
    possible_urls = [
        "https://terakoya.sejuku.net/account/profile",
        "https://terakoya.sejuku.net/mypage",
        "https://terakoya.sejuku.net/account",
        "https://terakoya.sejuku.net/mypage/profile",
    ]

    for i, url in enumerate(possible_urls, 1):
        print(f"\n試行 {i}/{len(possible_urls)}: {url}")
        try:
            driver.get(url)
            time.sleep(3)

            current_url = driver.current_url
            print(f"  → 実際のURL: {current_url}")

            page_source = driver.page_source.lower()

            # 404チェック
            if "404" in page_source or "not found" in page_source or "ページが見つかりません" in page_source:
                print(f"  → 404エラー")
                continue

            screenshot_name = f"page_{i}_{url.split('/')[-1]}.png"
            driver.save_screenshot(screenshot_name)
            print(f"  → スクリーンショット: {screenshot_name}")

            # 編集ボタンまたはtextareaを探す
            edit_elements = driver.find_elements(By.XPATH, "//*[contains(text(), '編集')]")
            textareas = driver.find_elements(By.TAG_NAME, 'textarea')

            if edit_elements:
                print(f"  ✅ 「編集」要素を発見！（{len(edit_elements)}個）")
                return url
            elif textareas:
                print(f"  ✅ textarea発見！（{len(textareas)}個）")
                return url
            else:
                print(f"  → 編集要素なし")

        except Exception as e:
            print(f"  → エラー: {str(e)}")

    print("\n❌ プロフィールページが見つかりません")
    return None

def edit_profile_complete(driver, new_bio):
    """
    プロフィール編集
    """
    print("\n" + "=" * 60)
    print("プロフィール編集処理...")
    print("=" * 60)

    try:
        # 編集ボタンを探す
        print("✓ 編集ボタンを検索中...")
        edit_button = None

        # 方法1: 提供されたクラス名
        try:
            edit_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sc-dTvVRJ.kunQxe'))
            )
            print("  → クラス名で発見")
        except:
            pass

        # 方法2: テキストで探す
        if not edit_button:
            try:
                edit_button = driver.find_element(By.XPATH, "//button[contains(text(), '編集')]")
                print("  → buttonテキストで発見")
            except:
                try:
                    edit_button = driver.find_element(By.XPATH, "//div[contains(text(), '編集')]")
                    print("  → divテキストで発見")
                except:
                    pass

        # 方法3: すべての要素から探す
        if not edit_button:
            all_elements = driver.find_elements(By.XPATH, "//*[contains(text(), '編集')]")
            for elem in all_elements:
                if elem.text.strip() == '編集' and elem.is_displayed():
                    edit_button = elem
                    print(f"  → {elem.tag_name}要素で発見")
                    break

        if edit_button:
            driver.execute_script("arguments[0].scrollIntoView(true);", edit_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", edit_button)
            print("✓ 編集ボタンをクリック")
            time.sleep(3)
            driver.save_screenshot("after_click_edit.png")
        else:
            print("⚠ 編集ボタンなし - 既に編集画面の可能性")

        # textareaを探す
        print("\n✓ 自己紹介文入力欄を検索中...")
        bio_textarea = None

        try:
            bio_textarea = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'textarea.sc-bPEjDU.vaHBV'))
            )
            print("  → クラス名で発見")
        except:
            textareas = driver.find_elements(By.TAG_NAME, 'textarea')
            if textareas:
                bio_textarea = textareas[0]
                print(f"  → textareaタグで発見（{len(textareas)}個中の1個目）")

        if not bio_textarea:
            print("❌ 自己紹介文入力欄が見つかりません")
            return False

        # 入力
        driver.execute_script("arguments[0].scrollIntoView(true);", bio_textarea)
        time.sleep(1)
        bio_textarea.clear()
        time.sleep(0.5)
        bio_textarea.send_keys(new_bio)
        print(f"✓ 自己紹介文入力完了: 「{new_bio}」")

        driver.save_screenshot("after_textarea_input.png")
        time.sleep(2)

        # 更新ボタンを探す
        print("\n✓ 更新ボタンを検索中...")
        update_button = None

        try:
            update_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sc-dTvVRJ.bHRgJQ'))
            )
            print("  → クラス名で発見")
        except:
            try:
                update_button = driver.find_element(By.XPATH, "//button[contains(text(), '更新')]")
                print("  → テキストで発見")
            except:
                buttons = driver.find_elements(By.TAG_NAME, "button")
                for btn in buttons:
                    if '更新' in btn.text:
                        update_button = btn
                        print("  → 検索で発見")
                        break

        if not update_button:
            print("❌ 更新ボタンが見つかりません")
            return False

        driver.execute_script("arguments[0].scrollIntoView(true);", update_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", update_button)
        print("✓ 更新ボタンをクリック")

        time.sleep(4)
        driver.save_screenshot("after_update_final.png")
        print("✓ プロフィール更新成功！")

        return True

    except Exception as e:
        print(f"❌ 編集エラー: {str(e)}")
        import traceback
        traceback.print_exc()
        try:
            driver.save_screenshot("error_edit.png")
        except:
            pass
        return False

def main():
    """
    メイン処理
    """
    EMAIL = "fts38579@gmail.com"
    PASSWORD = "Shogilover1205"
    NEW_BIO = "プログラミング学習中です！今はスクレイピングに挑戦しています！"

    driver = None

    try:
        print("=" * 60)
        print("🚀 侍テラコヤ プロフィール自動更新【完全版】")
        print("=" * 60)

        # Chrome起動
        print("\n⏳ Chromeを起動中...")
        driver = setup_chrome_driver()

        # ログイン
        if not login_to_terakoya(driver, EMAIL, PASSWORD):
            print("\n❌ ログイン失敗")
            return

        # プロフィールページに移動
        profile_url = navigate_to_profile(driver)

        if not profile_url:
            print("\n❌ プロフィールページが見つかりません")
            return

        print(f"\n✅ プロフィールページ発見: {profile_url}")

        # 編集処理
        if edit_profile_complete(driver, NEW_BIO):
            print("\n" + "=" * 60)
            print("🎉 すべての処理が正常に完了しました！")
            print("=" * 60)
            print("\n📸 保存されたスクリーンショット:")
            print("  - step1_top_page.png : トップページ")
            print("  - step2_login_page.png : ログインページ")
            print("  - step3_filled_form.png : フォーム入力後")
            print("  - step4_after_login.png : ログイン後")
            print("  - after_textarea_input.png : テキスト入力後")
            print("  - after_update_final.png : 更新後")
        else:
            print("\n❌ 編集処理失敗")

    except Exception as e:
        print(f"\n❌ 予期しないエラー: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        if driver:
            try:
                time.sleep(2)
                driver.quit()
                print("\n✓ ブラウザを終了しました")
            except:
                print("\n✓ ブラウザ終了済み")

if __name__ == "__main__":
    main()


🚀 侍テラコヤ プロフィール自動更新【完全版】

⏳ Chromeを起動中...
✓ Chromeドライバー初期化成功

ログイン処理を開始...
✓ トップページにアクセス: https://terakoya.sejuku.net/
  現在のURL: https://terakoya.sejuku.net/

✓ ヘッダーの「ログイン」ボタンを検索中...
  → divクラス名で発見
✓ ヘッダーの「ログイン」をクリック
  現在のURL: https://terakoya.sejuku.net/

✓ メールアドレス入力欄を検索中...
✓ メールアドレス入力完了: fts38579@gmail.com
✓ パスワード入力欄を検索中...
✓ パスワード入力完了: ********

✓ ログイン送信ボタンを検索中...
  → buttonタグで発見
✓ ログイン送信ボタンをクリック

⏳ ログイン処理待機中...
✓ ログイン後のURL: https://terakoya.sejuku.net/home
✓ ログイン成功！

プロフィールページに移動...

試行 1/4: https://terakoya.sejuku.net/account/profile
  → 実際のURL: https://terakoya.sejuku.net/account/profile
  → スクリーンショット: page_1_profile.png
  ✅ 「編集」要素を発見！（1個）

✅ プロフィールページ発見: https://terakoya.sejuku.net/account/profile

プロフィール編集処理...
✓ 編集ボタンを検索中...
  → クラス名で発見
✓ 編集ボタンをクリック

✓ 自己紹介文入力欄を検索中...
  → クラス名で発見
✓ 自己紹介文入力完了: 「プログラミング学習中です！今はスクレイピングに挑戦しています！」

✓ 更新ボタンを検索中...
  → クラス名で発見
✓ 更新ボタンをクリック
✓ プロフィール更新成功！

🎉 すべての処理が正常に完了しました！

📸 保存されたスクリーンショット:
  - step1_top_page.png : トップページ
  - step2_login_page.png :